### Import libraries and load datasets

In [80]:
import pandas as pd

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [81]:
x_train = pd.read_csv('scaled_feature_drop_na_dengue.csv').drop('Unnamed: 0', axis = 1)
y_train = pd.read_csv('label_drop_na_dengue.csv')['total_cases']

correlated_col = list(x_train.columns)

train_df = x_train.join(y_train)

train_df

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,iq,sj,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,total_cases
0,-0.122207,-0.253846,-0.095140,-0.328502,-0.814598,-0.808198,-1.190146,-1.926304,-1.171085,0.122086,-0.214733,-1.269681,-0.814598,-1.842236,-0.698244,-1.409262,-0.619643,-1.625800,-1.336941,-0.508735,False,True,False,False,False,True,False,False,False,False,False,False,False,False,4
1,0.215513,0.064732,-0.580536,-0.587307,-0.573640,-0.340090,-0.634981,-0.901462,-0.837468,0.315202,-0.530914,-0.721257,-0.573640,-0.946406,-0.769831,-0.405893,-0.862664,-0.445878,0.080211,-0.659830,False,True,False,False,False,False,True,False,False,False,False,False,False,False,5
2,-0.767304,0.319858,-0.649827,-0.407716,-0.302099,0.077752,-0.289418,0.087186,-0.958783,0.662810,-0.347412,-0.079473,-0.302099,0.025644,-0.789717,-0.405893,-0.810119,-0.189373,0.466707,0.009886,False,True,False,False,False,False,True,False,False,False,False,False,False,False,4
3,-0.079130,0.917244,0.295500,0.352911,-0.746481,0.228552,-0.011835,0.004322,-0.685825,0.546941,-0.621766,-0.314539,-0.746481,-0.090099,-0.753923,0.191619,-0.678756,0.374938,0.788787,-0.753753,False,True,False,False,False,False,True,False,False,False,False,False,False,False,3
4,0.403295,1.059203,0.613169,0.486861,-0.928126,0.618119,0.333727,0.345301,-0.534181,0.740057,-0.659995,-0.297707,-0.928126,0.263716,-0.590863,1.352821,0.516646,1.247054,1.175283,-0.717000,False,True,False,False,False,False,True,False,False,False,False,False,False,False,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1.449658,1.528992,0.682269,1.015140,0.178890,0.483027,1.211795,1.014877,1.831463,-0.418637,0.077611,0.840243,0.178890,1.104027,1.298248,1.108555,1.694533,1.452258,0.209043,-0.284136,True,False,False,False,False,False,True,False,False,False,False,False,False,False,5
1195,0.145949,0.215499,-0.934097,0.233295,0.901068,-0.253170,0.118459,0.766286,1.467517,-1.422839,3.722913,1.228563,0.901068,0.830197,0.649985,0.161555,1.035529,1.093151,-0.241869,-0.088122,True,False,False,False,False,False,False,True,False,False,False,False,False,False,8
1196,0.766413,0.096898,0.601843,0.327518,0.263225,-1.522404,-1.309110,0.131951,0.557655,-1.229723,0.203543,1.595745,0.263225,0.131036,0.657939,-2.232250,-0.619643,-0.189373,-1.852269,-0.684331,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1
1197,1.386571,0.923083,0.985157,1.400717,0.280138,-1.707763,-1.365760,0.013847,0.739627,-1.422839,0.467327,1.647808,0.280138,0.004001,0.761343,-1.416778,0.223269,-0.702382,-0.692781,-0.508735,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1


### Split train set and test set

In [82]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.3, random_state = 13)

x_train

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,iq,sj,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
199,-0.358184,-0.452906,-0.320450,-0.581460,-0.635270,0.845366,0.520670,0.392924,-0.685825,1.087665,-0.561723,-0.478949,-0.635270,0.320176,-0.785740,0.428369,-0.356917,0.118433,0.466707,0.052764,False,True,False,False,False,False,False,True,False,False,False,False,False,False
737,1.112906,1.336630,1.432964,0.704240,-0.566689,1.559572,2.135183,-3.112110,2.771654,-1.577331,-0.878129,-3.404071,-0.566689,-2.738065,2.718064,0.898111,2.138977,1.657462,-0.241869,0.326367,True,False,False,False,False,False,False,False,False,False,True,False,False,False
978,1.196240,1.191890,2.005535,0.950153,0.611455,0.662102,1.546028,0.310060,2.074093,-1.422839,-0.145470,-0.128991,0.611455,0.400161,1.978328,1.476833,2.047023,2.016568,-0.241869,0.471336,True,False,False,False,False,False,False,False,False,False,False,True,False,False
286,-0.933784,-0.726535,-0.866149,-0.551222,0.233337,0.853744,0.594315,0.846292,-0.534181,0.971795,0.027912,-0.032890,0.233337,0.776560,-0.741992,0.834226,-0.751006,0.118433,1.175283,0.254903,False,True,False,False,False,False,False,False,False,True,False,False,False,False
294,-1.105739,-0.424735,-0.049680,0.058772,1.199021,0.588797,0.152448,0.754857,-0.230893,0.778680,3.158464,0.129171,1.199021,0.736097,-0.916983,0.101428,-0.836392,-0.753683,0.466707,-0.706791,False,True,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0.754786,0.417074,0.750984,0.738822,0.847547,-2.750796,-3.172881,-0.490954,-0.655496,-2.118055,1.208756,2.123617,0.847547,-0.476849,-0.101682,-0.180417,0.253920,-0.035470,-0.048621,4.191522,True,False,False,False,False,False,True,False,False,False,False,False,False,False
742,0.598454,-0.003712,1.937970,-1.225451,-0.381801,-0.147401,0.826578,-0.473810,1.679819,-1.422839,-0.441862,-0.152087,-0.381801,-0.486259,2.324334,0.338178,1.403345,0.734044,-0.692781,-0.774171,True,False,False,False,False,False,False,False,False,False,False,True,False,False
74,-0.618793,-0.416864,-0.857128,-0.832552,-1.102357,-0.533826,-0.867244,-1.063380,-0.989112,-0.032406,-0.854291,-0.702468,-1.102357,-1.102612,-0.678358,-0.890667,-0.179577,-0.753683,-0.241869,-0.737419,False,True,False,False,True,False,False,False,False,False,False,False,False,False
176,0.309047,1.089031,-0.118716,-0.138081,-1.037716,0.954278,0.701949,-0.044253,-0.655496,1.126288,-0.860138,-0.986074,-1.037716,-0.151264,-0.745969,1.014607,-0.468576,0.374938,1.497363,-0.815008,False,True,False,False,False,False,False,False,True,False,False,False,False,False


In [83]:
x_train = x_train.astype(float)
x_test = x_test.astype(float)

y_train = pd.DataFrame(y_train).astype(float)
y_test = pd.DataFrame(y_test).astype(float)

x_train

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,iq,sj,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
199,-0.358184,-0.452906,-0.320450,-0.581460,-0.635270,0.845366,0.520670,0.392924,-0.685825,1.087665,-0.561723,-0.478949,-0.635270,0.320176,-0.785740,0.428369,-0.356917,0.118433,0.466707,0.052764,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
737,1.112906,1.336630,1.432964,0.704240,-0.566689,1.559572,2.135183,-3.112110,2.771654,-1.577331,-0.878129,-3.404071,-0.566689,-2.738065,2.718064,0.898111,2.138977,1.657462,-0.241869,0.326367,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
978,1.196240,1.191890,2.005535,0.950153,0.611455,0.662102,1.546028,0.310060,2.074093,-1.422839,-0.145470,-0.128991,0.611455,0.400161,1.978328,1.476833,2.047023,2.016568,-0.241869,0.471336,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
286,-0.933784,-0.726535,-0.866149,-0.551222,0.233337,0.853744,0.594315,0.846292,-0.534181,0.971795,0.027912,-0.032890,0.233337,0.776560,-0.741992,0.834226,-0.751006,0.118433,1.175283,0.254903,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
294,-1.105739,-0.424735,-0.049680,0.058772,1.199021,0.588797,0.152448,0.754857,-0.230893,0.778680,3.158464,0.129171,1.199021,0.736097,-0.916983,0.101428,-0.836392,-0.753683,0.466707,-0.706791,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,0.754786,0.417074,0.750984,0.738822,0.847547,-2.750796,-3.172881,-0.490954,-0.655496,-2.118055,1.208756,2.123617,0.847547,-0.476849,-0.101682,-0.180417,0.253920,-0.035470,-0.048621,4.191522,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
742,0.598454,-0.003712,1.937970,-1.225451,-0.381801,-0.147401,0.826578,-0.473810,1.679819,-1.422839,-0.441862,-0.152087,-0.381801,-0.486259,2.324334,0.338178,1.403345,0.734044,-0.692781,-0.774171,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
74,-0.618793,-0.416864,-0.857128,-0.832552,-1.102357,-0.533826,-0.867244,-1.063380,-0.989112,-0.032406,-0.854291,-0.702468,-1.102357,-1.102612,-0.678358,-0.890667,-0.179577,-0.753683,-0.241869,-0.737419,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,0.309047,1.089031,-0.118716,-0.138081,-1.037716,0.954278,0.701949,-0.044253,-0.655496,1.126288,-0.860138,-0.986074,-1.037716,-0.151264,-0.745969,1.014607,-0.468576,0.374938,1.497363,-0.815008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Neural Network - Mini-Batch stochastic gradient descent

In [84]:
para_dict = {
    'hidden_units' : [7, 8, 9],
    'batch_size' : [16, 32, 64],
    'learning_rate' : [[0.008, 0.01], [0.0008, 0.001]]
}

def create_model(columns, units = 8, activation = 'relu', loss = 'mae', optimizer = 'SGD', metrics = 'mae'):
    
    model = Sequential([
        Dense(units, input_shape = (len(columns), ), activation = activation),
        Dense(1, activation = activation)
    ])

    model.compile(
        optimizer = optimizer,
        loss = loss,
        metrics = metrics
    )

    return model

In [85]:
result_sgd_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = SGD(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, optimizer = optimizer)

            best_model = model.fit(
                x_train, y_train,
                validation_data = (x_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result_sgd_dict['units'].append(units)
            result_sgd_dict['batch_size'].append(b_size)
            result_sgd_dict['learning_rate'].append(rate)
            result_sgd_dict['minimum_mae_error'].append(min_mae)

Epoch 1/250
53/53 [==============================] - 1s 5ms/step - loss: 21.2206 - mae: 21.2206 - val_loss: 20.8805 - val_mae: 20.8805
Epoch 2/250
53/53 [==============================] - 0s 3ms/step - loss: 21.0612 - mae: 21.0612 - val_loss: 20.6419 - val_mae: 20.6419
Epoch 3/250
53/53 [==============================] - 0s 2ms/step - loss: 20.7127 - mae: 20.7127 - val_loss: 20.1825 - val_mae: 20.1825
Epoch 4/250
53/53 [==============================] - 0s 3ms/step - loss: 20.0845 - mae: 20.0845 - val_loss: 19.5132 - val_mae: 19.5132
Epoch 5/250
53/53 [==============================] - 0s 2ms/step - loss: 19.1245 - mae: 19.1245 - val_loss: 18.5423 - val_mae: 18.5423
Epoch 6/250
53/53 [==============================] - 0s 2ms/step - loss: 17.7347 - mae: 17.7347 - val_loss: 17.1992 - val_mae: 17.1992
Epoch 7/250
53/53 [==============================] - 0s 2ms/step - loss: 16.6427 - mae: 16.6427 - val_loss: 16.2308 - val_mae: 16.2308
Epoch 8/250
53/53 [==============================] - 0s

In [86]:
result_sgd_df = pd.DataFrame(result_sgd_dict)

result_sgd_df.to_csv('result33_adam.csv')

result_sgd_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,12.086053
1,7,16,0.010,12.094622
2,7,32,0.008,12.163536
3,7,32,0.010,12.152575
4,7,64,0.008,12.227609
5,7,64,0.010,12.304523
6,8,16,0.008,12.158232
7,8,16,0.010,12.089931
8,8,32,0.008,12.038342
9,8,32,0.010,12.264164


### Neural Network - Adam

In [87]:
result_adam_dict = {
    'units': [],
    'batch_size': [],
    'learning_rate': [],
    'minimum_mae_error': []
}

for units in para_dict['hidden_units']:
    for b_size in para_dict['batch_size']:
        for rate in para_dict['learning_rate'][0]:

            early_stopping = EarlyStopping(monitor = 'val_mae',
                               patience = 10,
                               restore_best_weights = True)

            optimizer = Adam(learning_rate = rate)

            model = create_model(columns = x_train.columns, units = units, loss = 'mse', optimizer = optimizer)

            best_model = model.fit(
                x_train, y_train,
                validation_data = (x_test, y_test),
                batch_size = b_size,
                epochs = 250,
                callbacks = [early_stopping]
            )

            min_mae = early_stopping.best

            result_adam_dict['units'].append(units)
            result_adam_dict['batch_size'].append(b_size)
            result_adam_dict['learning_rate'].append(rate)
            result_adam_dict['minimum_mae_error'].append(min_mae)

Epoch 1/250
53/53 [==============================] - 1s 5ms/step - loss: 1401.1251 - mae: 19.0983 - val_loss: 881.2196 - val_mae: 16.7015
Epoch 2/250
53/53 [==============================] - 0s 2ms/step - loss: 1058.4275 - mae: 15.7771 - val_loss: 591.4792 - val_mae: 14.7916
Epoch 3/250
53/53 [==============================] - 0s 3ms/step - loss: 894.8551 - mae: 15.9176 - val_loss: 543.9379 - val_mae: 14.3415
Epoch 4/250
53/53 [==============================] - 0s 2ms/step - loss: 855.7973 - mae: 16.0246 - val_loss: 520.7614 - val_mae: 14.3347
Epoch 5/250
53/53 [==============================] - 0s 2ms/step - loss: 829.4182 - mae: 15.5691 - val_loss: 509.1882 - val_mae: 14.4684
Epoch 6/250
53/53 [==============================] - 0s 2ms/step - loss: 810.0624 - mae: 15.7315 - val_loss: 494.0499 - val_mae: 13.8171
Epoch 7/250
53/53 [==============================] - 0s 2ms/step - loss: 794.0491 - mae: 15.1959 - val_loss: 485.5450 - val_mae: 13.5443
Epoch 8/250
53/53 [====================

In [88]:
result_adam_df = pd.DataFrame(result_adam_dict)

result_adam_df.to_csv('result33_adam.csv')

result_adam_df

,units,batch_size,learning_rate,minimum_mae_error
0,7,16,0.008,12.789749
1,7,16,0.010,13.004602
2,7,32,0.008,13.670699
3,7,32,0.010,13.124236
4,7,64,0.008,13.308177
5,7,64,0.010,13.106520
6,8,16,0.008,13.040756
7,8,16,0.010,13.137069
8,8,32,0.008,13.391715
9,8,32,0.010,12.994552
